# Подготовка данных

In [2]:
%matplotlib inline
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
# df = pd.read_csv(r"InternationalBifurca_DATA_2023-10-30_0629.csv", sep=',')
df = pd.read_csv(r"InternationalBifurca_DATA_2025-04-20_0932.csv", sep=',')

/var/folders/2x/tx689lxn54ngmttr2b89rvg00000gn/T/ipykernel_29184/3738330455.py:2: DtypeWarning: Columns (264) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"InternationalBifurca_DATA_2025-04-20_0932.csv", sep=',')


In [4]:
df = df.dropna(subset=['sex'])
df

,record_id,identifier,date,adhoc_pci,sex,age,weight,height,clinical_presentation,time_from_mi_symptoms_onse,...,time_to_death_f5,time_to_acs_f5,time_to_stroke_f5,time_to_pci_f5,time_to_cabg_f5,hospitalization_f5,bleeding_f5,angio_follow_f5,restenosis_f5,side_branch_restenosis_5
0,MNRI0001,MNRI-2018-0001,2018-02-01,1.0,2.0,77.0,84.0,165.0,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MNRI0002,MNRI-2018-0002,2018-01-24,0.0,1.0,68.0,81.0,171.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MNRI0003,MNRI-2018-0003,2018-01-24,0.0,1.0,62.0,74.0,180.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MNRI0004,MNRI-2018-0004,2018-01-30,1.0,1.0,67.0,84.0,167.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MNRI0005,MNRI-2018-0005,2018-01-30,0.0,1.0,57.0,103.0,174.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,TRCH-2019-0026,2019-03-11,1.0,1.0,67.0,90.0,174.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2140,TRCH0027,TRCH-2019-0027,2019-03-18,1.0,1.0,69.0,60.0,174.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2141,TRCH0028,TRCH-2019-0028,2019-03-19,0.0,2.0,81.0,50.0,160.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2142,TRCH0029,TRCH-2019-0029,2019-03-28,1.0,1.0,86.0,74.0,170.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
anyInf = df[df == np.inf].sum()
anyInf[anyInf != 0]

stent_distal_vessel_size    inf
sb_stent_sb_diametr         inf
dtype: object

In [6]:
patient_info_cols = ['record_id', 'date', 'sex', 'age', 'adhoc_pci', 'weight', 'height', 
                     'clinical_presentation', 'time_from_mi_symptoms_onse', 
                     'ccs_class', 'diabet', 'insulin_diabetes', 'hypertension', 'smoking', 
                     'dyslipidemia', 'anemia', 'atrial_fibrilation', 'oac_use', 'valvular_disease', 
                     'valvular_disease_was_previ', 'if_yes_what_type___1', 'if_yes_what_type___2', 
                     'if_yes_what_type___3', 'if_yes_what_type___4', 'if_yes_what_type___5', 
                     'if_yes_what_type___6', 'if_yes_what_type___7', 'ef', 'creatinine', 'ckd', 
                     'mi_history', 'cerebrovascular_disease', 'previously_treated_cerebro', 'previous_stroke_tia', 
                     'peripheral_artery_disease', 'previously_treated_periphe', 'copd', 'history_of_cancer', 
                     'previous_pci', 'previous_cabg']

intervention_cols = ['single_vessel', 'trifurcation', 'bifurcation_location', 
                  'lesion_ivolves', 'angle', 'calcium', 'trombosis', 
                  'total_trobotic_occlusion', 'restenosis_reocclusion', 'cto_bifurc', 
                  'medina_proximal', 'medina_distal', 'medina_side', 'mb_length_proximal', 
                  'sb_length', 'proximal_diametr', 'distal_diametr', 'side_diametr', 'stenosis_proximal', 
                  'stenosis_distal', 'timi_flow_main_branch', 'side_stenosis', 'timi_flow_side_branch', 
                  'major_lm', 'major_non_lm', 'minor_criteria', 'main_branch_rvd', 'def', 'def_2']

operation_cols = ['side_protection', 'main_predilatation', 'side_predilat', 
                  'stent_was_implated_from_lm', 'stent_number', 'stent_number_bif', 'stent_technique', 
                  'first_stent_impanted', 'provisional_2_stent_techni', 'stent_direction', 'defered_stenting', 
                  'stent_diameter', 'stent_length', 'stent_type___1', 'stent_type___2', 'stent_type___3', 
                  'stent_type___4', 'stent_type___5', 'stent_type___6', 'stent_type___7', 'stent_type___9', 
                  'stent_type___8', 'dstent2', 'stent_length2', 'stent_distal_vessel_size', 
                  'sb_stent_side_branch_diametr', 'sb_stent_sb_diametr', 'twostent_technique', 
                  'sb_dilatation', 'stent_postdilatation', 'proximal_optimization', 'pot', 
                  'pot_balloon_diametr', 'kissing_post', 'modified_kis', 'several_kissing']

new_cols = ['adverse_event_followup_f2_v2', 'angio_follow_f5', 'antiplatalet_drug_was_chan',
            'attempt_to_dilate_stenting', 'ballooon_size_for_postdila', 'complete_revascularisation',
            'currently_on_dialysis', 'followup_1_year_do_not_complete_if_2nd_bifurcation_complete',
            'identifier', 'ishemia_test___1', 'ishemia_test___2', 'ishemia_test___3', 'ishemia_test___4',
            'kissing_post_2stent___1', 'kissing_post_2stent___2', 'left_main_stent_direction',
            'main_branch_calcification', 'mb_stenosis_f2', 'medina_side_branch_2', 'myocardial_ischemia',
            'myocardial_ishemia_was_det', 'number_of_kissing', 'number_of_kissing_2',
            'other_lesions_in_main_bran', 'other_lesions_in_side_brach',
            'patient_information_do_not_complete_if_2nd_bifurca_complete', 'pot_2', 'pot_balloon_diametr_2',
            'pot_balloon_length', 'pot_balloon_length_2', 'pressure2', 'reson_for_change_stopped___1',
            'reson_for_change_stopped___2', 'reson_for_change_stopped___3', 'restenosis_f5', 'sb_length_2',
            'sb_stenosis_f2', 'side_branch_calcification_2', 'side_branch_restenosis',
            'side_branch_restenosis_3', 'side_branch_restenosis_5', 'stent_pressure', 'stent_type_2___1',
            'stent_type_2___2', 'stent_type_2___3', 'stent_type_2___4', 'stent_type_2___5', 'stent_type_2___6',
            'stent_type_2___7', 'stent_type_2___8', 'stent_type_2___9', 'thrombolysis', 'uncross_strategy___1',
            'uncross_strategy___2', 'uncross_strategy___3', 'uncross_strategy___4', 'uncross_strategy___5',
            'uncross_strategy___6', 'uncross_strategy___7', 'uncross_strategy___8', 'uncross_strategy___9']

In [7]:
non_vascular_deaths = ['MNRI1054', 'MNRI1191', 'MNRI1351', 'MNRI1352', 'MNRI1473', 'MNRI1670', 'MNRI0637', 'MNRI0656', 'MNRI0751', 'MNRI0758',
                      'MNRI0805', 'MNRI0818', 'MNRI1054', 'MNRI0087', 'MNRI1191', 'MNRI0108', 'MNRI0307', 'MNRI0215', 'MNRI0322', 'MNRI0293',
                      'MNRI0156', 'MNRI0215', 'MNRI0488', 'MNRI0612', 'MNRI0708', 'MNRI0767', 'MNRI0772', 'MNRI0786', 'MNRI1105', 'MNRI1186',
                      'MNRI1462', 'MNRI1633']

In [8]:
df = df[~df['record_id'].isin(non_vascular_deaths)]

In [9]:
df

,record_id,identifier,date,adhoc_pci,sex,age,weight,height,clinical_presentation,time_from_mi_symptoms_onse,...,time_to_death_f5,time_to_acs_f5,time_to_stroke_f5,time_to_pci_f5,time_to_cabg_f5,hospitalization_f5,bleeding_f5,angio_follow_f5,restenosis_f5,side_branch_restenosis_5
0,MNRI0001,MNRI-2018-0001,2018-02-01,1.0,2.0,77.0,84.0,165.0,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MNRI0002,MNRI-2018-0002,2018-01-24,0.0,1.0,68.0,81.0,171.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MNRI0003,MNRI-2018-0003,2018-01-24,0.0,1.0,62.0,74.0,180.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MNRI0004,MNRI-2018-0004,2018-01-30,1.0,1.0,67.0,84.0,167.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MNRI0005,MNRI-2018-0005,2018-01-30,0.0,1.0,57.0,103.0,174.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,TRCH-2019-0026,2019-03-11,1.0,1.0,67.0,90.0,174.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2140,TRCH0027,TRCH-2019-0027,2019-03-18,1.0,1.0,69.0,60.0,174.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2141,TRCH0028,TRCH-2019-0028,2019-03-19,0.0,2.0,81.0,50.0,160.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2142,TRCH0029,TRCH-2019-0029,2019-03-28,1.0,1.0,86.0,74.0,170.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
patient_present_cols = patient_info_cols + intervention_cols + operation_cols + new_cols
patient_present_df = pd.DataFrame({col_name: df[col_name] for col_name in patient_present_cols})

In [11]:
patient_present_df

,record_id,date,sex,age,adhoc_pci,weight,height,clinical_presentation,time_from_mi_symptoms_onse,ccs_class,...,thrombolysis,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___5,uncross_strategy___6,uncross_strategy___7,uncross_strategy___8,uncross_strategy___9
0,MNRI0001,2018-02-01,2.0,77.0,1.0,84.0,165.0,5.0,4.0,NaN,...,0.0,0,0,0,0,0,0,0,0,0
1,MNRI0002,2018-01-24,1.0,68.0,0.0,81.0,171.0,1.0,NaN,1.0,...,NaN,0,0,0,0,0,0,0,0,0
2,MNRI0003,2018-01-24,1.0,62.0,0.0,74.0,180.0,4.0,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
3,MNRI0004,2018-01-30,1.0,67.0,1.0,84.0,167.0,2.0,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
4,MNRI0005,2018-01-30,1.0,57.0,0.0,103.0,174.0,1.0,NaN,2.0,...,NaN,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,2019-03-11,1.0,67.0,1.0,90.0,174.0,2.0,NaN,NaN,...,0.0,0,0,0,0,0,0,0,0,0
2140,TRCH0027,2019-03-18,1.0,69.0,1.0,60.0,174.0,3.0,1.0,NaN,...,1.0,0,0,0,0,0,0,0,0,0
2141,TRCH0028,2019-03-19,2.0,81.0,0.0,50.0,160.0,2.0,NaN,NaN,...,0.0,0,0,0,0,0,0,0,0,0
2142,TRCH0029,2019-03-28,1.0,86.0,1.0,74.0,170.0,3.0,1.0,NaN,...,0.0,0,0,0,0,0,0,0,0,0


In [12]:
# columns_to_preserve = ['valvular_disease', 'previous_stroke_tia', 'twostent_technique']
columns_to_preserve = []
def remove_columns_with_nan_threshold(df, threshold=250):
    nan_counts = df.isnull().sum()
    
    columns_to_drop = [col for col in nan_counts[nan_counts > threshold].index 
                      if col not in columns_to_preserve]
    
    print(columns_to_drop)
    
    df_dropped = df.drop(columns=columns_to_drop)
    
    return df_dropped

In [13]:
patient_present_df = remove_columns_with_nan_threshold(patient_present_df, 500)

['time_from_mi_symptoms_onse', 'ccs_class', 'insulin_diabetes', 'valvular_disease', 'valvular_disease_was_previ', 'previously_treated_cerebro', 'previous_stroke_tia', 'previously_treated_periphe', 'lesion_ivolves', 'total_trobotic_occlusion', 'sb_length', 'timi_flow_main_branch', 'timi_flow_side_branch', 'stent_was_implated_from_lm', 'first_stent_impanted', 'provisional_2_stent_techni', 'dstent2', 'stent_length2', 'sb_stent_side_branch_diametr', 'twostent_technique', 'proximal_optimization', 'pot', 'pot_balloon_diametr', 'several_kissing', 'adverse_event_followup_f2_v2', 'angio_follow_f5', 'antiplatalet_drug_was_chan', 'attempt_to_dilate_stenting', 'ballooon_size_for_postdila', 'complete_revascularisation', 'left_main_stent_direction', 'main_branch_calcification', 'mb_stenosis_f2', 'medina_side_branch_2', 'myocardial_ischemia', 'myocardial_ishemia_was_det', 'number_of_kissing', 'number_of_kissing_2', 'other_lesions_in_main_bran', 'other_lesions_in_side_brach', 'pot_2', 'pot_balloon_dia

In [14]:
# patient_present_df = patient_present_df.drop(['record_id', 'date'], axis = 1)
patient_present_df = patient_present_df.drop(['date'], axis = 1)
patient_present_df

,record_id,sex,age,adhoc_pci,weight,height,clinical_presentation,diabet,hypertension,smoking,...,stent_type_2___9,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___5,uncross_strategy___6,uncross_strategy___7,uncross_strategy___8,uncross_strategy___9
0,MNRI0001,2.0,77.0,1.0,84.0,165.0,5.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,MNRI0002,1.0,68.0,0.0,81.0,171.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,MNRI0003,1.0,62.0,0.0,74.0,180.0,4.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,MNRI0004,1.0,67.0,1.0,84.0,167.0,2.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,MNRI0005,1.0,57.0,0.0,103.0,174.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,1.0,67.0,1.0,90.0,174.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2140,TRCH0027,1.0,69.0,1.0,60.0,174.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2141,TRCH0028,2.0,81.0,0.0,50.0,160.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2142,TRCH0029,1.0,86.0,1.0,74.0,170.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
patient_present_df.replace(to_replace = [np.inf, -np.inf], value= None, inplace=True)

In [16]:
cols_to_drop = patient_present_df.columns[patient_present_df.nunique() <= 1]
patient_present_df = patient_present_df.drop(cols_to_drop, axis=1)
patient_present_df

,record_id,sex,age,adhoc_pci,weight,height,clinical_presentation,diabet,hypertension,smoking,...,stent_type_2___5,stent_type_2___6,stent_type_2___7,stent_type_2___8,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___8,uncross_strategy___9
0,MNRI0001,2.0,77.0,1.0,84.0,165.0,5.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,MNRI0002,1.0,68.0,0.0,81.0,171.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,MNRI0003,1.0,62.0,0.0,74.0,180.0,4.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,MNRI0004,1.0,67.0,1.0,84.0,167.0,2.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,MNRI0005,1.0,57.0,0.0,103.0,174.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,1.0,67.0,1.0,90.0,174.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2140,TRCH0027,1.0,69.0,1.0,60.0,174.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2141,TRCH0028,2.0,81.0,0.0,50.0,160.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2142,TRCH0029,1.0,86.0,1.0,74.0,170.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
numerical = ['age', 'weight', 'height', 'ef', 'creatinine', 'ckd', 'angle', 'mb_length_proximal', 
            'proximal_diametr', 'distal_diametr', 'side_diametr', 'stenosis_proximal', 
            'stenosis_distal', 'side_stenosis', 'minor_criteria', 'main_branch_rvd', 
            'stent_diameter', 'stent_length', 'stent_distal_vessel_size', 'sb_stent_sb_diametr',
            'ballooon_size_for_postdila', 
            'left_main_stent_direction',
            'mb_stenosis_f2',
            'myocardial_ischemia',
            'number_of_kissing_2',
            'pot_balloon_diametr_2',
            'pot_balloon_length',
            'pot_balloon_length_2',
            'pressure2',
            'sb_length_2',
            'sb_stenosis_f2',
            'stent_pressure']

categorical = ['sex', 'clinical_presentation', 'bifurcation_location', 'stent_number', 
              'stent_number_bif', 'stent_technique', 'stent_direction']

binary = ['diabet', 'adhoc_pci', 'hypertension', 'smoking', 'dyslipidemia', 'anemia', 
         'atrial_fibrilation', 'oac_use', 'if_yes_what_type___1', 'if_yes_what_type___2',
         'if_yes_what_type___3', 'if_yes_what_type___4', 'if_yes_what_type___6', 
         'mi_history', 'cerebrovascular_disease', 'peripheral_artery_disease', 'copd', 
         'history_of_cancer', 'previous_pci', 'previous_cabg', 'single_vessel', 'trifurcation',
         'calcium', 'trombosis', 'restenosis_reocclusion', 'cto_bifurc', 
         'medina_proximal', 'medina_distal', 'medina_side', 'major_lm', 'major_non_lm',
         'def', 'def_2', 'side_protection', 'main_predilatation', 'side_predilat',
         'defered_stenting', 'stent_type___1', 'stent_type___2', 'stent_type___3', 'stent_type___4',
         'stent_type___5', 'stent_type___6', 'stent_type___7', 'stent_type___9', 'stent_type___8', 
         'sb_dilatation', 'stent_postdilatation', 'kissing_post', 'modified_kis',
         'currently_on_dialysis',
         'ishemia_test___1',
         'ishemia_test___2',
         'ishemia_test___3',
         'kissing_post_2stent___1',
         'kissing_post_2stent___2',
         'reson_for_change_stopped___1',
         'reson_for_change_stopped___2',
         'reson_for_change_stopped___3',
         'stent_type_2___1',
         'stent_type_2___3',
         'stent_type_2___4',
         'stent_type_2___5',
         'stent_type_2___6',
         'stent_type_2___7',
         'stent_type_2___8',
         'uncross_strategy___1',
         'uncross_strategy___2',
         'uncross_strategy___3',
         'uncross_strategy___4',
         'uncross_strategy___8',
         'uncross_strategy___9']

In [18]:
without_second_bif = df
adverse_events = without_second_bif['event_type_followup_f2___1'] \
| without_second_bif['event_type_followup_f2___2'] \
| without_second_bif['event_type_followup_f2_v2___1'] \
| without_second_bif['event_type_followup_f2_v2___2']
print(sum(adverse_events))
print(len(adverse_events))

158
2044


In [19]:
combined = zip(without_second_bif['event_type_followup_f2___1'],  
               without_second_bif['event_type_followup_f2___2'] * 2,
               without_second_bif['event_type_followup_f2_v2___1'] * 3, 
               without_second_bif['event_type_followup_f2_v2___2'] * 4)

combined_adverse_events = np.array([max(t) for t in combined])

In [20]:
unique, counts = np.unique(combined_adverse_events, return_counts=True)
value_counts = dict(zip(unique, counts))
print(value_counts)

{0: 1886, 1: 53, 2: 18, 3: 51, 4: 36}


In [21]:
id_col = patient_present_df['record_id']
patient_present_df = patient_present_df.drop(['record_id', 'identifier'], axis=1)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(patient_present_df, combined_adverse_events, test_size=0.4, stratify=combined_adverse_events, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

In [23]:
y_train[y_train != 0] = 1
y_test[y_test != 0] = 1
y_val[y_val != 0] = 1

In [24]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

imputer_categorical = SimpleImputer(strategy='most_frequent')
X_train[categorical + binary] = imputer_categorical.fit_transform(X_train[categorical + binary])
X_test[categorical + binary] = imputer_categorical.transform(X_test[categorical + binary])
X_val[categorical + binary] = imputer_categorical.transform(X_val[categorical + binary])

imputer = IterativeImputer(random_state=0)
columns = list(X_train.columns)
X_train = pd.DataFrame(data = imputer.fit_transform(X_train), columns = columns)
X_test = pd.DataFrame(data = imputer.transform(X_test), columns = columns)
X_val = pd.DataFrame(data = imputer.transform(X_val), columns = columns)

X_train

,sex,age,adhoc_pci,weight,height,clinical_presentation,diabet,hypertension,smoking,dyslipidemia,...,stent_type_2___5,stent_type_2___6,stent_type_2___7,stent_type_2___8,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___8,uncross_strategy___9
0,1.0,41.0,0.0,100.000000,177.000000,5.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,56.0,1.0,81.000000,172.000000,4.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,55.0,1.0,87.541718,172.066825,3.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,79.0,0.0,61.000000,160.000000,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,61.0,1.0,84.000000,179.000000,2.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,2.0,69.0,0.0,111.000000,159.000000,1.0,2.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1222,1.0,57.0,0.0,96.000000,178.000000,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1223,1.0,63.0,0.0,80.000000,162.000000,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1224,1.0,75.0,0.0,165.000000,71.000000,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
df_for_ohe = pd.concat([X_train[categorical], X_test[categorical], X_val[categorical]], ignore_index=True)
ohe.fit(df_for_ohe)

OneHotEncoder(sparse_output=False)

In [26]:
def process_ohe(dataframe, cat_cols, encoder):
    encoded_columns = pd.DataFrame(encoder.transform(dataframe[cat_cols]))

    encoded_columns.columns = encoder.get_feature_names_out(cat_cols)

    dataframe = pd.concat([dataframe, encoded_columns], axis=1)

    dataframe.drop(cat_cols, axis=1, inplace=True)

    return dataframe

In [27]:
X_train = process_ohe(X_train, categorical, ohe)
X_test = process_ohe(X_test, categorical, ohe)
X_val = process_ohe(X_val, categorical, ohe)
X_train

,age,adhoc_pci,weight,height,diabet,hypertension,smoking,dyslipidemia,anemia,atrial_fibrilation,...,stent_number_bif_3.0,stent_number_bif_4.0,stent_technique_0.0,stent_technique_1.0,stent_direction_1.0,stent_direction_2.0,stent_direction_3.0,stent_direction_4.0,stent_direction_5.0,stent_direction_6.0
0,41.0,0.0,100.000000,177.000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,56.0,1.0,81.000000,172.000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,55.0,1.0,87.541718,172.066825,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,79.0,0.0,61.000000,160.000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,61.0,1.0,84.000000,179.000000,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,69.0,0.0,111.000000,159.000000,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1222,57.0,0.0,96.000000,178.000000,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1223,63.0,0.0,80.000000,162.000000,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1224,75.0,0.0,165.000000,71.000000,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [28]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

feature_selector = SelectKBest(f_classif, k=40)
X_feature_selection = feature_selector.fit_transform(X_train, y_train)
X_feature_selection.shape

strong_cols = []

feature_scores = feature_selector.scores_
features = X_train.columns
features_scores_sorted = sorted(zip(features, feature_scores), key=lambda x: x[1], reverse=True)
for col in features_scores_sorted[:30]:
    strong_cols.append(col[0])

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [74] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [29]:
X_train = X_train[strong_cols]
X_test = X_test[strong_cols]
X_val = X_val[strong_cols]
X_train

,age,anemia,ef,cerebrovascular_disease,ckd,peripheral_artery_disease,if_yes_what_type___1,creatinine,single_vessel,calcium,...,minor_criteria,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc,side_diametr,major_lm,main_predilatation,mi_history
0,41.0,0.0,49.0,0.0,91.130343,0.0,0.0,90.0,0.0,0.0,...,1.0,11.0,0.0,1.0,0.0,0.0,2.4,0.0,0.0,1.0
1,56.0,0.0,68.0,0.0,83.132035,0.0,0.0,89.0,1.0,0.0,...,0.0,18.0,1.0,1.0,1.0,0.0,3.0,0.0,1.0,1.0
2,55.0,0.0,70.0,0.0,78.364633,0.0,0.0,94.0,1.0,0.0,...,2.0,33.0,1.0,0.0,1.0,0.0,2.4,0.0,1.0,0.0
3,79.0,0.0,73.0,1.0,64.542788,0.0,0.0,96.0,0.0,1.0,...,3.0,38.0,0.0,0.0,0.0,0.0,3.1,0.0,1.0,0.0
4,61.0,0.0,62.0,0.0,87.329657,0.0,0.0,83.0,0.0,0.0,...,2.0,28.0,1.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,69.0,0.0,60.0,0.0,69.608441,0.0,0.0,97.0,1.0,0.0,...,0.0,32.0,0.0,1.0,0.0,0.0,2.7,0.0,1.0,0.0
1222,57.0,0.0,39.0,0.0,53.707072,0.0,0.0,127.0,0.0,0.0,...,2.0,23.0,0.0,0.0,0.0,0.0,2.4,0.0,1.0,0.0
1223,63.0,1.0,71.0,0.0,86.111321,0.0,0.0,83.0,1.0,0.0,...,2.0,18.0,0.0,1.0,0.0,0.0,2.2,0.0,0.0,0.0
1224,75.0,0.0,69.0,1.0,68.979615,1.0,0.0,93.0,1.0,0.0,...,0.0,18.0,0.0,1.0,0.0,0.0,2.4,0.0,1.0,0.0


In [30]:
X_train.columns

Index(['age', 'anemia', 'ef', 'cerebrovascular_disease', 'ckd',
       'peripheral_artery_disease', 'if_yes_what_type___1', 'creatinine',
       'single_vessel', 'calcium', 'stent_type___3', 'medina_side',
       'atrial_fibrilation', 'height', 'def', 'history_of_cancer',
       'side_stenosis', 'stent_type___5', 'side_predilat', 'stent_diameter',
       'minor_criteria', 'stent_length', 'adhoc_pci', 'previous_pci',
       'stent_type___4', 'cto_bifurc', 'side_diametr', 'major_lm',
       'main_predilatation', 'mi_history'],
      dtype='object')

In [31]:
columns_to_preserve

[]

In [32]:
corr_matrix = X_train.corr().abs()

upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

threshold = 0.4

to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]

to_drop

['ckd',
 'creatinine',
 'side_stenosis',
 'side_predilat',
 'minor_criteria',
 'side_diametr',
 'major_lm',
 'mi_history']

In [33]:
X_train = X_train.drop(columns=to_drop)
X_test = X_test.drop(columns=to_drop)
X_val = X_val.drop(columns=to_drop)

X_train

,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,medina_side,...,def,history_of_cancer,stent_type___5,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc,main_predilatation
0,41.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,4.00,11.0,0.0,1.0,0.0,0.0,0.0
1,56.0,0.0,68.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,2.75,18.0,1.0,1.0,1.0,0.0,1.0
2,55.0,0.0,70.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.75,33.0,1.0,0.0,1.0,0.0,1.0
3,79.0,0.0,73.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,3.50,38.0,0.0,0.0,0.0,0.0,1.0
4,61.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,4.00,28.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,69.0,0.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.50,32.0,0.0,1.0,0.0,0.0,1.0
1222,57.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.50,23.0,0.0,0.0,0.0,0.0,1.0
1223,63.0,1.0,71.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2.75,18.0,0.0,1.0,0.0,0.0,0.0
1224,75.0,0.0,69.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.00,18.0,0.0,1.0,0.0,0.0,1.0


In [34]:
X_train.columns

Index(['age', 'anemia', 'ef', 'cerebrovascular_disease',
       'peripheral_artery_disease', 'if_yes_what_type___1', 'single_vessel',
       'calcium', 'stent_type___3', 'medina_side', 'atrial_fibrilation',
       'height', 'def', 'history_of_cancer', 'stent_type___5',
       'stent_diameter', 'stent_length', 'adhoc_pci', 'previous_pci',
       'stent_type___4', 'cto_bifurc', 'main_predilatation'],
      dtype='object')

In [35]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)

(1226, 22)
(409, 22)
(409, 22)
(1226,)


In [36]:
print(X_train.columns)

Index(['age', 'anemia', 'ef', 'cerebrovascular_disease',
       'peripheral_artery_disease', 'if_yes_what_type___1', 'single_vessel',
       'calcium', 'stent_type___3', 'medina_side', 'atrial_fibrilation',
       'height', 'def', 'history_of_cancer', 'stent_type___5',
       'stent_diameter', 'stent_length', 'adhoc_pci', 'previous_pci',
       'stent_type___4', 'cto_bifurc', 'main_predilatation'],
      dtype='object')


# edge case

In [37]:
import importlib
import edge_case
importlib.reload(edge_case)
from edge_case import generate_edge_cases

edge_cases = generate_edge_cases(num_samples=100)
edge_cases


,age,height,clinical_presentation,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,medina_side,...,restenosis_reocclusion,adhoc_pci,main_predilatation,stent_diameter,stent_length,mortality,smoking,dyslipidemia,anemia,atrial_fibrilation
0,91.951277,166.536366,5,27.554474,1,1,1,1,1,1,...,1,1,1,2.473314,28.725651,1,0,1,1,1
1,86.789651,168.496341,5,24.896786,1,1,1,1,1,1,...,1,0,0,2.298368,32.434836,1,1,1,1,1
2,98.029493,151.920128,5,29.546548,1,1,1,1,1,1,...,1,1,0,2.393204,30.471564,1,1,1,1,1
3,94.300111,146.666875,5,23.358203,1,1,1,1,1,1,...,1,0,1,2.147390,31.417380,1,1,1,1,0
4,85.919670,152.259192,5,17.147886,1,1,1,1,1,1,...,1,1,1,2.610013,29.217105,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,97.288244,164.427135,5,20.356745,1,1,0,1,1,1,...,1,0,1,2.084591,29.314002,1,1,1,1,1
96,89.307110,167.752409,5,22.546855,1,1,0,1,1,1,...,1,1,1,2.660724,37.617807,1,0,1,1,0
97,98.273245,152.073710,5,28.903042,1,1,1,1,0,0,...,1,1,0,2.616955,37.655861,1,1,0,1,1
98,87.279906,147.590719,5,19.626283,1,1,1,1,1,1,...,1,1,1,2.064636,31.240584,1,1,1,1,1


In [38]:
edge_cases.columns

Index(['age', 'height', 'clinical_presentation', 'ef',
       'cerebrovascular_disease', 'peripheral_artery_disease',
       'if_yes_what_type___1', 'single_vessel', 'calcium', 'medina_side',
       'trifurcation', 'cto_bifurc', 'def', 'history_of_cancer',
       'previous_pci', 'previous_stroke_tia', 'side_diametr', 'stent_type___3',
       'stent_type___4', 'stent_type___5', 'restenosis_reocclusion',
       'adhoc_pci', 'main_predilatation', 'stent_diameter', 'stent_length',
       'mortality', 'smoking', 'dyslipidemia', 'anemia', 'atrial_fibrilation'],
      dtype='object')

In [39]:
edge_case_cols = set(edge_cases.columns)
x_train_cols = set(X_train.columns)

common_cols = edge_case_cols.intersection(x_train_cols)
only_in_edge_cases = edge_case_cols - x_train_cols
only_in_x_train = x_train_cols - edge_case_cols

print(f"COMPARISON OF COLUMNS:")
print(f"\nTotal columns in edge_cases: {len(edge_case_cols)}")
print(f"Total columns in X_train: {len(x_train_cols)}")
print(f"Number of common columns: {len(common_cols)}")

print("\n--- COMMON COLUMNS ---")
for col in sorted(common_cols):
    print(f"- {col}")

print("\n--- COLUMNS ONLY IN EDGE CASES ---")
for col in sorted(only_in_edge_cases):
    print(f"- {col}")

print("\n--- COLUMNS ONLY IN X_TRAIN ---")
for col in sorted(only_in_x_train):
    print(f"- {col}")

COMPARISON OF COLUMNS:

Total columns in edge_cases: 30
Total columns in X_train: 22
Number of common columns: 22

--- COMMON COLUMNS ---
- adhoc_pci
- age
- anemia
- atrial_fibrilation
- calcium
- cerebrovascular_disease
- cto_bifurc
- def
- ef
- height
- history_of_cancer
- if_yes_what_type___1
- main_predilatation
- medina_side
- peripheral_artery_disease
- previous_pci
- single_vessel
- stent_diameter
- stent_length
- stent_type___3
- stent_type___4
- stent_type___5

--- COLUMNS ONLY IN EDGE CASES ---
- clinical_presentation
- dyslipidemia
- mortality
- previous_stroke_tia
- restenosis_reocclusion
- side_diametr
- smoking
- trifurcation

--- COLUMNS ONLY IN X_TRAIN ---


In [40]:
edge_cases_target = edge_cases['mortality']
edge_cases_features = edge_cases.copy().drop('mortality', axis=1)

# Syntetic data

In [41]:
X_train_with_ID = X_train.copy().reset_index(drop=False)
X_train_with_ID

,index,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,...,def,history_of_cancer,stent_type___5,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc,main_predilatation
0,0,41.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,4.00,11.0,0.0,1.0,0.0,0.0,0.0
1,1,56.0,0.0,68.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,2.75,18.0,1.0,1.0,1.0,0.0,1.0
2,2,55.0,0.0,70.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2.75,33.0,1.0,0.0,1.0,0.0,1.0
3,3,79.0,0.0,73.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.50,38.0,0.0,0.0,0.0,0.0,1.0
4,4,61.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,4.00,28.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,1221,69.0,0.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,3.50,32.0,0.0,1.0,0.0,0.0,1.0
1222,1222,57.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,3.50,23.0,0.0,0.0,0.0,0.0,1.0
1223,1223,63.0,1.0,71.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,2.75,18.0,0.0,1.0,0.0,0.0,0.0
1224,1224,75.0,0.0,69.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,3.00,18.0,0.0,1.0,0.0,0.0,1.0


In [42]:
X_train_with_ID_and_target = X_train_with_ID.copy()
X_train_with_ID_and_target['target'] = y_train
X_train_with_ID_and_target

,index,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,...,history_of_cancer,stent_type___5,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc,main_predilatation,target
0,0,41.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,4.00,11.0,0.0,1.0,0.0,0.0,0.0,0
1,1,56.0,0.0,68.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,2.75,18.0,1.0,1.0,1.0,0.0,1.0,0
2,2,55.0,0.0,70.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,2.75,33.0,1.0,0.0,1.0,0.0,1.0,0
3,3,79.0,0.0,73.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,3.50,38.0,0.0,0.0,0.0,0.0,1.0,0
4,4,61.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.00,28.0,1.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,1221,69.0,0.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,3.50,32.0,0.0,1.0,0.0,0.0,1.0,1
1222,1222,57.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,3.50,23.0,0.0,0.0,0.0,0.0,1.0,0
1223,1223,63.0,1.0,71.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,2.75,18.0,0.0,1.0,0.0,0.0,0.0,0
1224,1224,75.0,0.0,69.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,3.00,18.0,0.0,1.0,0.0,0.0,1.0,0


In [43]:
from synthcity.plugins import Plugins

# Print the list of available plugins
print(Plugins().list())

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode


[2025-05-26T19:44:05.077747+0400][29184][CRITICAL] load failed: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.
[2025-05-26T19:44:05.078335+0400][29184][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2025-05-26T19:44:05.078545+0400][29184][CRITICAL] module plugin_great load failed
[2025-05-26T19:44:05.186586+0400][29184][CRITICAL] module disabled: /Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/synthcity/plugins/generic/plugin_goggle.py


['privbayes', 'survival_ctgan', 'rtvae', 'radialgan', 'tvae', 'decaf', 'pategan', 'survival_nflow', 'aim', 'dpgan', 'uniform_sampler', 'fflows', 'image_cgan', 'bayesian_network', 'survae', 'ddpm', 'nflow', 'arf', 'ctgan', 'adsgan', 'timegan', 'timevae', 'survival_gan', 'marginal_distributions', 'image_adsgan', 'dummy_sampler']


# Оптимизация RF на auc-roc для 10 фолдов

In [44]:
# X_train_new = pd.concat([X_train, syntetic_minority_dropped])
# X_train_new

In [45]:
# y_train_new = np.concatenate((y_train, syntetic_target))
# y_train_new

In [46]:
edge_cases_features_names = [feature for feature in edge_cases.columns if feature in X_train.columns]
edge_cases_features = pd.DataFrame(edge_cases, columns=edge_cases_features_names)
edge_cases_features

,age,height,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,medina_side,cto_bifurc,...,previous_pci,stent_type___3,stent_type___4,stent_type___5,adhoc_pci,main_predilatation,stent_diameter,stent_length,anemia,atrial_fibrilation
0,91.951277,166.536366,27.554474,1,1,1,1,1,1,1,...,0,1,1,1,1,1,2.473314,28.725651,1,1
1,86.789651,168.496341,24.896786,1,1,1,1,1,1,1,...,1,1,0,0,0,0,2.298368,32.434836,1,1
2,98.029493,151.920128,29.546548,1,1,1,1,1,1,1,...,1,0,1,0,1,0,2.393204,30.471564,1,1
3,94.300111,146.666875,23.358203,1,1,1,1,1,1,1,...,1,1,0,1,0,1,2.147390,31.417380,1,0
4,85.919670,152.259192,17.147886,1,1,1,1,1,1,1,...,1,0,1,1,1,1,2.610013,29.217105,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,97.288244,164.427135,20.356745,1,1,0,1,1,1,1,...,1,1,1,0,0,1,2.084591,29.314002,1,1
96,89.307110,167.752409,22.546855,1,1,0,1,1,1,1,...,0,1,1,0,1,1,2.660724,37.617807,1,0
97,98.273245,152.073710,28.903042,1,1,1,1,0,0,1,...,1,1,1,0,1,0,2.616955,37.655861,1,1
98,87.279906,147.590719,19.626283,1,1,1,1,1,1,1,...,1,0,1,1,1,1,2.064636,31.240584,1,1


In [47]:
X_train_folds = pd.concat([X_train, X_val, edge_cases_features])
y_train_k_fold = np.concatenate((y_train, y_val, edge_cases_target), axis=0)

In [48]:
from sklearn.preprocessing import StandardScaler

scaller = StandardScaler()
scaller.fit(X_train_folds)

StandardScaler()

In [49]:
scaled_features = scaller.transform(X_train_folds)

In [50]:
train_df = pd.DataFrame(data=scaled_features, columns=X_train_folds.columns)
train_df

,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,medina_side,...,def,history_of_cancer,stent_type___5,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc,main_predilatation
0,-2.094997,-0.330658,-0.405060,-0.450766,-0.376095,-0.277436,-0.983423,-0.576685,1.287233,-0.753193,...,-0.244674,-0.298563,-0.219860,1.481974,-1.676793,-0.868503,1.139304,-0.431184,-0.371086,-1.332666
1,-0.817866,-0.330658,1.057244,-0.450766,-0.376095,-0.277436,1.016857,-0.576685,-0.776860,1.327682,...,-0.244674,3.349375,-0.219860,-0.826967,-0.839506,1.151407,1.139304,2.319197,-0.371086,0.750375
2,-0.903008,-0.330658,1.211171,-0.450766,-0.376095,-0.277436,1.016857,-0.576685,-0.776860,-0.753193,...,-0.244674,-0.298563,-0.219860,-0.826967,0.954681,1.151407,-0.877729,2.319197,-0.371086,0.750375
3,1.140401,-0.330658,1.442061,2.218446,-0.376095,-0.277436,-0.983423,1.734050,-0.776860,-0.753193,...,-0.244674,-0.298563,-0.219860,0.558398,1.552743,-0.868503,-0.877729,-0.431184,-0.371086,0.750375
4,-0.392156,-0.330658,0.595464,-0.450766,-0.376095,-0.277436,-0.983423,-0.576685,-0.776860,-0.753193,...,-0.244674,3.349375,-0.219860,1.481974,0.356619,1.151407,-0.877729,-0.431184,-0.371086,-1.332666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730,2.697499,3.024268,-2.609540,2.218446,2.658903,-0.277436,1.016857,1.734050,1.287233,1.327682,...,4.087063,3.349375,-0.219860,-2.056079,0.513790,-0.868503,1.139304,2.319197,2.694792,0.750375
1731,2.017969,3.024268,-2.440982,2.218446,2.658903,-0.277436,1.016857,1.734050,1.287233,1.327682,...,4.087063,3.349375,-0.219860,-0.991874,1.507028,1.151407,-0.877729,2.319197,2.694792,0.750375
1732,2.781364,3.024268,-1.951789,2.218446,2.658903,3.604433,1.016857,-0.576685,1.287233,-0.753193,...,4.087063,-0.298563,-0.219860,-1.072720,1.511580,1.151407,1.139304,2.319197,2.694792,-1.332666
1733,1.845369,3.024268,-2.665759,2.218446,2.658903,3.604433,1.016857,1.734050,-0.776860,1.327682,...,-0.244674,3.349375,4.548351,-2.092938,0.744233,1.151407,1.139304,2.319197,2.694792,0.750375


In [51]:
X_train_folds.columns

Index(['age', 'anemia', 'ef', 'cerebrovascular_disease',
       'peripheral_artery_disease', 'if_yes_what_type___1', 'single_vessel',
       'calcium', 'stent_type___3', 'medina_side', 'atrial_fibrilation',
       'height', 'def', 'history_of_cancer', 'stent_type___5',
       'stent_diameter', 'stent_length', 'adhoc_pci', 'previous_pci',
       'stent_type___4', 'cto_bifurc', 'main_predilatation'],
      dtype='object')

In [52]:
if all(isinstance(cat, int) for cat in categorical):
    # Converting indices to column names
    categorical_names = [X_train_folds.columns[i] for i in categorical if i < len(X_train_folds.columns)]
else:
    # Already column names
    categorical_names = categorical

# Find intersection with actual columns in X_train_folds
valid_categorical_features = [cat for cat in categorical_names if cat in X_train_folds.columns]

# Print information for debugging
print(f"Total categorical features specified: {len(categorical_names)}")
print(f"Categorical features present in training data: {len(valid_categorical_features)}")
print(f"Missing categorical features: {set(categorical_names) - set(valid_categorical_features)}")

Total categorical features specified: 7
Categorical features present in training data: 0
Missing categorical features: {'stent_direction', 'stent_number_bif', 'sex', 'clinical_presentation', 'stent_number', 'bifurcation_location', 'stent_technique'}


In [53]:
valid_categorical_features

[]

In [54]:
numerical_features = [feature for feature in numerical if feature in X_train_folds.columns]
numerical_features

['age', 'height', 'ef', 'stent_diameter', 'stent_length']

# RF + edge cases

In [90]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import pickle
import sys
import warnings
import json
import os

# Suppress warnings
warnings.filterwarnings("ignore")

def objective(params):
    # Convert parameters to proper format
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth']) if params['max_depth'] is not None else None
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_auc_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X_train_folds, y_train_k_fold):
        # Get train and test data for this fold
        X_train = X_train_folds.iloc[train_index]
        X_test = X_train_folds.iloc[test_index]
        y_train = y_train_k_fold[train_index]
        y_test = y_train_k_fold[test_index]
        
        # Apply StandardScaler only to numerical features
        scaler = StandardScaler()
        X_train_scaled = X_train.copy()
        X_test_scaled = X_test.copy()
        
        # Scale only numerical features
        if len(numerical_features) > 0:
            X_train_scaled[numerical_features] = scaler.fit_transform(X_train[numerical_features])
            X_test_scaled[numerical_features] = scaler.transform(X_test[numerical_features])
        
        # Initialize classifier with current parameters
        classifier = RandomForestClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split'],
            min_samples_leaf=params['min_samples_leaf'],
            max_features=params['max_features'],
            bootstrap=params['bootstrap'],
            random_state=42
        )
        
        # Fit the model on scaled data
        classifier.fit(X_train_scaled, y_train)
        
        # Make predictions on scaled data
        predictions = classifier.predict_proba(X_test_scaled)[:, 1]
        y_pred = classifier.predict(X_test_scaled)
        
        # Calculate metrics
        roc_auc = roc_auc_score(y_test, predictions)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        
        roc_auc_scores.append(roc_auc)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        accuracy_scores.append(accuracy)

    mean_metrics = {
        'roc_auc': np.mean(roc_auc_scores),
        'f1': np.mean(f1_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'accuracy': np.mean(accuracy_scores),
        'roc_auc_scores': roc_auc_scores,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'accuracy_scores': accuracy_scores
    }
    
    print(f"Trial completed - AUC: {mean_metrics['roc_auc']:.4f}, Params: {params}")
    
    return {'loss': -mean_metrics['roc_auc'], 'status': STATUS_OK, 'params': params, 'mean_metrics': mean_metrics}

# Define the search space for Random Forest
space = {
    'n_estimators': hp.quniform('n_estimators', 100, 500, 100),  # 100, 200, 300, 400, 500
    'max_depth': hp.choice('max_depth', [None, 10, 20, 30, 40, 50]),  # None for unlimited depth
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 2),  # 2, 4, 6, 8, 10
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 5, 1),  # 1, 2, 3, 4, 5
    'max_features': hp.choice('max_features', ['sqrt', 'log2']),
    'bootstrap': hp.choice('bootstrap', [True, False])
}

# Run hyperparameter optimization with limited trials
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)

# Extract best parameters
best_trial = trials.best_trial['result']
best_params = best_trial['params']
best_metrics = best_trial['mean_metrics']

print("Hyperparameter optimization completed.")
print(f"Best parameters: {best_params}")
print(f"Best mean AUC: {best_metrics['roc_auc']:.4f}")

# Train final model with best hyperparameters on full dataset
# For the final model, scale the entire dataset
scaler_final = StandardScaler()

# Scale numerical features for the final model
X_train_final = X_train_folds.copy()
if len(numerical_features) > 0:
    X_train_final[numerical_features] = scaler_final.fit_transform(X_train_folds[numerical_features])

y_train_final = y_train_k_fold

best_model = RandomForestClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']) if best_params['max_depth'] is not None else None,
    min_samples_split=int(best_params['min_samples_split']),
    min_samples_leaf=int(best_params['min_samples_leaf']),
    max_features=best_params['max_features'],
    bootstrap=best_params['bootstrap'],
    random_state=42
)

# Fit final model on scaled data
best_model.fit(X_train_final, y_train_final)

# Save the best model and scaler
with open('model_RF_edge_cases_CV.pkl', 'wb') as f:
    pickle.dump(best_model, f)

with open('scaler_RF_edge_cases_CV.pkl', 'wb') as f:
    pickle.dump(scaler_final, f)

# Save metrics and best parameters to files
metrics_and_params = {
    'best_parameters': {
        'n_estimators': int(best_params['n_estimators']),
        'max_depth': int(best_params['max_depth']) if best_params['max_depth'] is not None else None,
        'min_samples_split': int(best_params['min_samples_split']),
        'min_samples_leaf': int(best_params['min_samples_leaf']),
        'max_features': best_params['max_features'],
        'bootstrap': best_params['bootstrap']
    },
    'evaluation_metrics': best_metrics
}

with open('scores_RF_edge_cases_CV.json', 'w') as f:
    json.dump(metrics_and_params, f, default=lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

print("Best parameters, model, scaler, numerical features list, and evaluation metrics saved.")

Trial completed - AUC: 0.8241, Params: {'bootstrap': True, 'max_depth': 50, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 200}
Trial completed - AUC: 0.8246, Params: {'bootstrap': False, 'max_depth': 50, 'max_features': 'log2', 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 300}
Trial completed - AUC: 0.8290, Params: {'bootstrap': True, 'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 300}
Trial completed - AUC: 0.8302, Params: {'bootstrap': True, 'max_depth': 40, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 6, 'n_estimators': 400}
Trial completed - AUC: 0.8214, Params: {'bootstrap': False, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 500}
Trial completed - AUC: 0.8320, Params: {'bootstrap': True, 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 5, 'min_samples_split': 6, 'n_estimat

In [94]:
X_test_scaled = X_test.copy()
if len(numerical_features) > 0:
    X_test_scaled[numerical_features] = scaler_final.transform(X_test[numerical_features])

test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_labels = best_model.predict(X_test_scaled)

test_metrics = {
    'roc_auc': roc_auc_score(y_test, test_predictions),
    'f1': f1_score(y_test, test_pred_labels, zero_division=0),
    'precision': precision_score(y_test, test_pred_labels, zero_division=0),
    'recall': recall_score(y_test, test_pred_labels, zero_division=0),
    'accuracy': accuracy_score(y_test, test_pred_labels)
}

print("\nTest dataset evaluation:")
print(f"ROC AUC: {test_metrics['roc_auc']:.4f}")
print(f"F1 Score: {test_metrics['f1']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")

metrics = {
    'test_evaluation_metrics': test_metrics,
    'best_parameters': {
        'n_estimators': int(best_params['n_estimators']),
        'max_depth': int(best_params['max_depth']) if best_params['max_depth'] is not None else None,
        'min_samples_split': int(best_params['min_samples_split']),
        'min_samples_leaf': int(best_params['min_samples_leaf']),
        'max_features': best_params['max_features'],
        'bootstrap': best_params['bootstrap']
    },
    'evaluation_metrics': best_metrics
}

with open('scores_RF_edge_cases_CV.json', 'w') as f:
    json.dump(metrics, f, default=lambda x: x.tolist() if isinstance(x, np.ndarray) else x)


Test dataset evaluation:
ROC AUC: 0.7840
F1 Score: 0.0000
Precision: 0.0000
Recall: 0.0000
Accuracy: 0.9242


# RF + edge_cases + syntetic

In [ ]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import pickle
import sys
import warnings
import json
import os

warnings.filterwarnings("ignore")

import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import GenericDataLoader

log.add(sink=sys.stderr, level="INFO")

def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth']) if params['max_depth'] is not None else None
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_auc_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X_train_folds, y_train_k_fold):
        not_scaled_X_train = X_train_folds.iloc[train_index].copy()
        X_test = X_train_folds.iloc[test_index].copy()
        y_train = y_train_k_fold[train_index]
        y_test = y_train_k_fold[test_index]
        
        train_df = not_scaled_X_train.copy().reset_index(drop=True)
        train_df['target'] = y_train
        
        loader = GenericDataLoader(
            train_df,
            target_column="target",
        )
        
        syn_model = Plugins().get("arf")
        syn_model.fit(loader)
        
        synthetic_data = syn_model.generate(count=10000).dataframe()
        
        minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
        if len(minority_synthetic_data) > 600:
            minority_synthetic_data = minority_synthetic_data.sample(n=600)
        
        syntetic_minority_features = minority_synthetic_data.drop('target', axis=1)
        syntetic_target = minority_synthetic_data['target']
        
        X_train_combined = pd.concat([not_scaled_X_train, syntetic_minority_features])
        y_train_combined = np.concatenate((y_train, syntetic_target.values), axis=0)
        
        X_train_scaled = X_train_combined.copy()
        X_test_scaled = X_test.copy()
        
        if len(numerical_features) > 0:
            scaler = StandardScaler()
            X_train_scaled[numerical_features] = scaler.fit_transform(X_train_combined[numerical_features])
            X_test_scaled[numerical_features] = scaler.transform(X_test[numerical_features])
        
        classifier = RandomForestClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split'],
            min_samples_leaf=params['min_samples_leaf'],
            max_features=params['max_features'],
            bootstrap=params['bootstrap'],
            random_state=42
        )
        
        classifier.fit(X_train_scaled, y_train_combined)
        
        predictions = classifier.predict_proba(X_test_scaled)[:, 1]
        y_pred = classifier.predict(X_test_scaled)
        
        roc_auc = roc_auc_score(y_test, predictions)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        
        roc_auc_scores.append(roc_auc)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        accuracy_scores.append(accuracy)

    mean_metrics = {
        'roc_auc': np.mean(roc_auc_scores),
        'f1': np.mean(f1_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'accuracy': np.mean(accuracy_scores),
        'roc_auc_scores': roc_auc_scores,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'accuracy_scores': accuracy_scores
    }
    
    print(f"Trial completed - AUC: {mean_metrics['roc_auc']:.4f}, Params: {params}")
    
    return {'loss': -mean_metrics['roc_auc'], 'status': STATUS_OK, 'params': params, 'mean_metrics': mean_metrics}

space = {
    'n_estimators': hp.quniform('n_estimators', 100, 500, 100),
    'max_depth': hp.choice('max_depth', [None, 20, 30, 40]),
    'min_samples_split': hp.quniform('min_samples_split', 2, 9, 3),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 5, 1),
    'max_features': hp.choice('max_features', ['sqrt', 'log2']),
    'bootstrap': hp.choice('bootstrap', [True, False])
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

best_trial = trials.best_trial['result']
best_params = best_trial['params']
best_metrics = best_trial['mean_metrics']

print("Hyperparameter optimization completed.")
print(f"Best parameters: {best_params}")
print(f"Best mean AUC: {best_metrics['roc_auc']:.4f}")

train_full = X_train_folds.copy().reset_index(drop=True)
train_full['target'] = y_train_k_fold

full_loader = GenericDataLoader(
    train_full,
    target_column="target"
)

syn_model = Plugins().get("arf")
syn_model.fit(full_loader)
synthetic_data = syn_model.generate(count=10000).dataframe()

minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
if len(minority_synthetic_data) > 600:
    minority_synthetic_data = minority_synthetic_data.sample(n=600)
syntetic_minority_features = minority_synthetic_data.drop('target', axis=1)
syntetic_target = minority_synthetic_data['target']

X_train_final = pd.concat([X_train_folds, syntetic_minority_features])
y_train_final = np.concatenate((y_train_k_fold, syntetic_target.values), axis=0)

scaler_final = StandardScaler()
X_train_final_scaled = X_train_final.copy()
if len(numerical_features) > 0:
    X_train_final_scaled[numerical_features] = scaler_final.fit_transform(X_train_final[numerical_features])

best_model = RandomForestClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']) if best_params['max_depth'] is not None else None,
    min_samples_split=int(best_params['min_samples_split']),
    min_samples_leaf=int(best_params['min_samples_leaf']),
    max_features=best_params['max_features'],
    bootstrap=best_params['bootstrap'],
    random_state=42
)

best_model.fit(X_train_final_scaled, y_train_final)

with open('model_RF_arf_edge_cases_CV.pkl', 'wb') as f:
    pickle.dump(best_model, f)

with open('scaler_RF_arf_edge_cases_CV.pkl', 'wb') as f:
    pickle.dump(scaler_final, f)

if 'X_test' in globals() and 'y_test' in globals():
    X_test_scaled = X_test.copy()
    if len(numerical_features) > 0:
        X_test_scaled[numerical_features] = scaler_final.transform(X_test[numerical_features])
    
    test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
    test_pred_labels = best_model.predict(X_test_scaled)
    
    test_metrics = {
        'roc_auc': roc_auc_score(y_test, test_predictions),
        'f1': f1_score(y_test, test_pred_labels, zero_division=0),
        'precision': precision_score(y_test, test_pred_labels, zero_division=0),
        'recall': recall_score(y_test, test_pred_labels, zero_division=0),
        'accuracy': accuracy_score(y_test, test_pred_labels)
    }
    
    print("\nTest dataset evaluation:")
    print(f"ROC AUC: {test_metrics['roc_auc']:.4f}")
    print(f"F1 Score: {test_metrics['f1']:.4f}")
    print(f"Precision: {test_metrics['precision']:.4f}")
    print(f"Recall: {test_metrics['recall']:.4f}")
    print(f"Accuracy: {test_metrics['accuracy']:.4f}")

try:
    with open('scores_RF_arf_edge_cases_CV.json', 'r') as f:
        existing_metrics = json.load(f)
except (FileNotFoundError, json.JSONDecodeError):
    existing_metrics = {}

metrics_to_save = {
    'best_parameters': {
        'n_estimators': int(best_params['n_estimators']),
        'max_depth': int(best_params['max_depth']) if best_params['max_depth'] is not None else None,
        'min_samples_split': int(best_params['min_samples_split']),
        'min_samples_leaf': int(best_params['min_samples_leaf']),
        'max_features': best_params['max_features'],
        'bootstrap': best_params['bootstrap']
    },
    'cv_evaluation_metrics': best_metrics,
    'numerical_features': numerical_features
}

if 'X_test' in globals() and 'y_test' in globals():
    metrics_to_save['test_evaluation_metrics'] = test_metrics

existing_metrics.update(metrics_to_save)

with open('scores_RF_arf_edge_cases_CV.json', 'w') as f:
    json.dump(existing_metrics, f, default=lambda x: x.tolist() if isinstance(x, np.ndarray) else x, indent=2)

print("\nBest parameters, model, scaler, and evaluation metrics saved to scores_RF_arf_edge_cases_CV.json")

In [100]:
X_test_scaled = X_test.copy()
if len(numerical_features) > 0:
    X_test_scaled[numerical_features] = scaler_final.transform(X_test[numerical_features])

test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_labels = best_model.predict(X_test_scaled)

test_metrics = {
    'roc_auc': roc_auc_score(y_test, test_predictions),
    'f1': f1_score(y_test, test_pred_labels, zero_division=0),
    'precision': precision_score(y_test, test_pred_labels, zero_division=0),
    'recall': recall_score(y_test, test_pred_labels, zero_division=0),
    'accuracy': accuracy_score(y_test, test_pred_labels)
}

print("\nTest dataset evaluation:")
print(f"ROC AUC: {test_metrics['roc_auc']:.4f}")
print(f"F1 Score: {test_metrics['f1']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")

metrics = {
    'test_evaluation_metrics': test_metrics,
    'best_parameters': {
        'n_estimators': int(best_params['n_estimators']),
        'max_depth': int(best_params['max_depth']) if best_params['max_depth'] is not None else None,
        'min_samples_split': int(best_params['min_samples_split']),
        'min_samples_leaf': int(best_params['min_samples_leaf']),
        'max_features': best_params['max_features'],
        'bootstrap': best_params['bootstrap']
    },
    'evaluation_metrics': best_metrics
}

with open('scores_RF_arf_edge_cases_CV.json', 'w') as f:
    json.dump(metrics, f, default=lambda x: x.tolist() if isinstance(x, np.ndarray) else x)


Test dataset evaluation:
ROC AUC: 0.7681
F1 Score: 0.1951
Precision: 0.4000
Recall: 0.1290
Accuracy: 0.9193


# RF + TVAE

In [1]:
best_parameters = {
  "n_estimators": 300,
  "max_depth": 10,
  "min_samples_split": 8,
  "min_samples_leaf": 5,
  "max_features": "log2"
}

In [54]:
from sdv.single_table import GaussianCopulaSynthesizer, CTGANSynthesizer, TVAESynthesizer, CopulaGANSynthesizer
from sdv.metadata import Metadata

X_train_with_ID = X_train.copy().reset_index(drop=False)
X_train_with_ID_and_target = X_train_with_ID.copy()
X_train_with_ID_and_target['target'] = y_train

metadata = Metadata.detect_from_dataframe(data=X_train_with_ID_and_target)

In [56]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier

# Define the best parameters for RandomForest
best_params = {
  "n_estimators": 300,
  "max_depth": 10,
  "min_samples_split": 8,
  "min_samples_leaf": 5,
  "max_features": "log2"
}

# Train the model with the best hyperparameters on X_train_folds
if not isinstance(X_train_folds, pd.DataFrame):
    X_train_df = pd.DataFrame(X_train_folds)
else:
    X_train_df = X_train_folds.copy()

# Add target column to the training data
X_train_df['target'] = y_train_k_fold
X_train_df['index'] = np.arange(1, len(X_train_df) + 1)

# Generate synthetic data using TVAE
synthesizer = TVAESynthesizer(metadata, epochs=500)
synthesizer.fit(X_train_df)
synthetic_data = synthesizer.sample(num_rows=10000)
minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
if len(minority_synthetic_data) > 600:
    minority_synthetic_data = minority_synthetic_data.sample(n=600)

# Extract features and target from synthetic data
syntetic_minority_dropped = minority_synthetic_data.copy().drop(['target', 'index'], axis=1)
syntetic_target = minority_synthetic_data['target']

# Combine original and synthetic data
X_train_combined = pd.concat([X_train_df.drop(['target', 'index'], axis=1), syntetic_minority_dropped])
y_train_combined = np.concatenate((y_train_k_fold, syntetic_target), axis=0)

# Scale the combined training data
X_train_combined_scaled = scaller.fit_transform(X_train_combined)

# Train the model with the best hyperparameters on scaled combined data
best_model = RandomForestClassifier(**best_params, verbose=0, n_jobs=-1)
best_model.fit(X_train_combined_scaled, y_train_combined)

# Scale the test data
X_test_scaled = scaller.transform(X_test)

# Make predictions
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_labels = best_model.predict(X_test_scaled)

# Calculate evaluation metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_labels, zero_division=0)
test_precision = precision_score(y_test, test_pred_labels, zero_division=0)
test_recall = recall_score(y_test, test_pred_labels, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_labels)

# Create dictionary for test metrics
test_metrics = {
    'test_roc_auc': float(test_roc_auc),
    'test_f1': float(test_f1),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'test_accuracy': float(test_accuracy)
}

# Print the metrics
print("Test Metrics:")
for metric_name, metric_value in test_metrics.items():
    print(f"{metric_name}: {metric_value:.4f}")

# Save the model and metrics
from joblib import dump
model_filename = 'model_RF_TVAE_test.joblib'
dump(best_model, model_filename)

# Save the test metrics to a file
metrics_and_params = {
    'best_parameters': best_params,
    'test_metrics': test_metrics
}

with open('test_scores_RF_TVAE.json', 'w') as f:
    json.dump(metrics_and_params, f, indent=4)

Test Metrics:
test_roc_auc: 0.7360
test_f1: 0.2381
test_precision: 0.4545
test_recall: 0.1613
test_accuracy: 0.9218


# RF + GAUSSIAN COPULA

In [61]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier

# Define the best parameters for RandomForest
best_params = {
  "n_estimators": 300,
  "max_depth": 10,
  "min_samples_split": 8,
  "min_samples_leaf": 5,
  "max_features": "log2"
}

# Train the model with the best hyperparameters on X_train_folds
if not isinstance(X_train_folds, pd.DataFrame):
    X_train_df = pd.DataFrame(X_train_folds)
else:
    X_train_df = X_train_folds.copy()

# Add target column to the training data
X_train_df['target'] = y_train_k_fold
X_train_df['index'] = np.arange(1, len(X_train_df) + 1)

# Generate synthetic data using TVAE
synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.fit(X_train_df)
synthetic_data = synthesizer.sample(num_rows=10000)
minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
if len(minority_synthetic_data) > 600:
    minority_synthetic_data = minority_synthetic_data.sample(n=600)

# Extract features and target from synthetic data
syntetic_minority_dropped = minority_synthetic_data.copy().drop(['target', 'index'], axis=1)
syntetic_target = minority_synthetic_data['target']

# Combine original and synthetic data
X_train_combined = pd.concat([X_train_df.drop(['target', 'index'], axis=1), syntetic_minority_dropped])
y_train_combined = np.concatenate((y_train_k_fold, syntetic_target), axis=0)

# Scale the combined training data
X_train_combined_scaled = scaller.fit_transform(X_train_combined)

# Train the model with the best hyperparameters on scaled combined data
best_model = RandomForestClassifier(**best_params, verbose=0, n_jobs=-1)
best_model.fit(X_train_combined_scaled, y_train_combined)

# Scale the test data
X_test_scaled = scaller.transform(X_test)

# Make predictions
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_labels = best_model.predict(X_test_scaled)

# Calculate evaluation metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_labels, zero_division=0)
test_precision = precision_score(y_test, test_pred_labels, zero_division=0)
test_recall = recall_score(y_test, test_pred_labels, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_labels)

# Create dictionary for test metrics
test_metrics = {
    'test_roc_auc': float(test_roc_auc),
    'test_f1': float(test_f1),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'test_accuracy': float(test_accuracy)
}

# Print the metrics
print("Test Metrics:")
for metric_name, metric_value in test_metrics.items():
    print(f"{metric_name}: {metric_value:.4f}")

# Save the model and metrics
from joblib import dump
model_filename = 'model_RF_GaussianCopula_test.joblib'
dump(best_model, model_filename)

# Save the test metrics to a file
metrics_and_params = {
    'best_parameters': best_params,
    'test_metrics': test_metrics
}

with open('test_scores_RF_GaussianCopula.json', 'w') as f:
    json.dump(metrics_and_params, f, indent=4)

Test Metrics:
test_roc_auc: 0.7630
test_f1: 0.2553
test_precision: 0.3750
test_recall: 0.1935
test_accuracy: 0.9144


In [60]:
Test Metrics:
test_roc_auc: 0.7646
test_f1: 0.2609
test_precision: 0.4000
test_recall: 0.1935
test_accuracy: 0.9169

SyntaxError: invalid syntax (593634353.py, line 1)